# Optimization with JuMP

JulaOpt is a collection of Julia optimization tools, and it's one of the best parts of the Julia ecosystem. Some of the packages in JuliaOpt are:

![](juliaopt.png)

For today, we're just going to show off JuMP, a modeling tool designed to make it easy to efficiently specify and solve optimizations in Julia. JuMP is similar to `yalmip` (in MATLAB), `pyomo` (in Python), or AMPL. It distinguishes itself by being fast and easy to use and taking advantage of Julia's expressiveness. 

In [1]:
using JuMP

Creating a model in JuMP is easy:

In [2]:
model = Model()

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

We add variables to the model with the `@variable` macro. Having a macro is useful because it can create variables inside the model and also, conveniently, create matching local variables in Julia:

In [3]:
@variable(model, x[1:2])

model

Feasibility problem with:
 * 0 linear constraints
 * 2 variables
Solver is default solver

In [4]:
typeof(x)

Array{JuMP.Variable,1}

Let's add a simple objective:

In [5]:
@objective(model, Min, sum(x.^2))
model

Minimization problem with:
 * 0 linear constraints
 * 2 variables
Solver is default solver

Now we can solve the model. JuMP will automatically try to choose a good solver for your problem based on what you have installed. You can always override this choice if you want. 

In [6]:
solve(model)

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds
Optimal objective 0.00000000e+00


:Optimal

And we can retrieve the solution:

In [7]:
getvalue(x)

2-element Array{Float64,1}:
 -0.0
 -0.0

That's a pretty boring model. Let's constrain the optimization:

In [8]:
@constraints model begin
    x[1] >= 2 * x[2]
    x[2] >= 1
end
model

Minimization problem with:
 * 2 linear constraints
 * 2 variables
Solver is default solver

In [9]:
solve(model)
getvalue(x)

Optimize a model with 2 rows, 2 columns and 3 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds
Optimal objective 5.00000000e+00


2-element Array{Float64,1}:
 2.0
 1.0

That was easy. But it's still not a very interesting optimization. How about something more robotics-y? 

## Model-Predictive Control in JuMP

Let's implement a very simple model-predictive control (MPC) optimization in JuMP. Specifically, we'll write an optimization that tries to find a sequence of control inputs for a very simple model robot in order to optimize an objective function. 

More concretely, our model will be a brick, sliding frictionlessly in two dimensions. Our input will be the acceleration of the brick, and we'll try to minimize the brick's final velocity and distance from the origin. 

In [71]:
model = Model()

# Define our constant parameters
Δt = 0.1
num_time_steps = 20
max_acceleration = 0.5

# Define our decision variables
@variables model begin
    position[1:2, 1:num_time_steps]
    velocity[1:2, 1:num_time_steps]
    -max_acceleration <= acceleration[1:2, 1:num_time_steps] <= max_acceleration
end

# Add dynamics constraints
@constraint(model, [i=2:num_time_steps, j=1:2],
            velocity[j, i] == velocity[j, i - 1] + acceleration[j, i - 1] * Δt)
@constraint(model, [i=2:num_time_steps, j=1:2],
            position[j, i] == position[j, i - 1] + velocity[j, i - 1] * Δt)

# Cost function: minimize final position and final velocity
@variable(model, slack >= 0)
objective_vars = vcat(position[:, end], velocity[:, end])
@constraint(model, objective_vars' * objective_vars .<= slack^2)
@objective(model, Min, slack)

# Initial conditions:
@constraint(model, position[:, 1] .== [1, 0])
@constraint(model, velocity[:, 1] .== [0, -1])

solve(model)

Optimize a model with 80 rows, 121 columns and 232 nonzeros
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-01, 5e-01]
  RHS range        [1e+00, 1e+00]
Presolve removed 74 rows and 76 columns
Presolve time: 0.00s
Presolved: 6 rows, 45 columns, 82 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+01
 Factor NZ  : 2.100e+01
 Factor Ops : 9.100e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.38876792e+00 -6.40190000e-01  1.71e+01 1.00e-01  1.37e-01     0s
   1   2.89543765e-01 -2.99130110e-01  3.00e+00 5.03e-05  2.53e-02     0s
   2   2.53374510e-01  6.50375538e-01  1.84e+00 6.00e-08  1.08e-02     0s
   3   7.11400631e-01  7.93178323e-01  8.51e

:Optimal

In [72]:
# Extract the solution from the model
q = getvalue(position)
v = getvalue(velocity)
u = getvalue(acceleration)

2×20 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  …  0.499999  0.5       0.5       -0.5
  0.5   0.5   0.5   0.5   0.5     0.5       0.499999  0.499999  -0.5

### Drawing the Result

We can draw the output of the optimization using the `Plots.jl` package

In [73]:
using Plots
gr()

Plots.GRBackend()

In [74]:
# The @gif macro creates an animate plot, which lets us draw the
# optimized trajectory of the brick as a function of time
@gif for i = 1:num_time_steps
    plot(q[1, :], q[2, :], xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
    plot!([q[1, i]], [q[2, i]], marker=(:hex, 6))
end

INFO: Saved animation to /Users/rdeits/locomotion/explorations/Robocon2017Tutorial.jl/notebooks/tmp.gif


Plots.AnimatedGif("/Users/rdeits/locomotion/explorations/Robocon2017Tutorial.jl/notebooks/tmp.gif")

## Running the MPC Controller

In a real application, we wouldn't just run the MPC optimization once. Instead, we might run the optimization at every time step using the robot's current state. 

To do that, let's wrap the MPC problem in a function called `run_mpc()` that takes the robot's current position and velocity as input:

In [63]:
if Pkg.installed("Gurobi") !== nothing
    # Prefer Gurobi if it's available
    using Gurobi
    solver = GurobiSolver(OutputFlag=0)
else
    using ECOS
    solver = ECOSSolver(verbose=0)
end

# run_mpc() takes the robot's current position and velocity
# and returns an optimized trajectory of position, velocity, 
# and acceleration. 
function run_mpc(initial_position, initial_velocity)
    
    model = Model(solver=solver)

    Δt = 0.1
    num_time_steps = 10
    max_acceleration = 0.5

    @variables model begin
        position[1:2, 1:num_time_steps]
        velocity[1:2, 1:num_time_steps]
        -max_acceleration <= acceleration[1:2, 1:num_time_steps] <= max_acceleration
    end

    # Dynamics constraints
    @constraint(model, [i=2:num_time_steps, j=1:2],
                velocity[j, i] == velocity[j, i - 1] + acceleration[j, i - 1] * Δt)
    @constraint(model, [i=2:num_time_steps, j=1:2],
                position[j, i] == position[j, i - 1] + velocity[j, i - 1] * Δt)

    # Cost function: minimize final position and final velocity
    @variable(model, slack >= 0)
    objective_vars = vcat(position[:, end], velocity[:, end])
    @constraint(model, objective_vars' * objective_vars .<= slack^2)
    @objective(model, Min, slack)

    # Initial conditions:
    @constraint(model, position[:, 1] .== initial_position)
    @constraint(model, velocity[:, 1] .== initial_velocity)

    solve(model)
    return getvalue(position), getvalue(velocity), getvalue(acceleration)
end

run_mpc (generic function with 1 method)

We can demonstrate this by repeatedly running the MPC program, applying its planned acceleration to the brick, and then simulating one step forward in time:

In [69]:
# The robot's starting position and velocity
q = [1.0, 0.0]
v = [0.0, -1.0]

@gif for i in 1:80
    # Plot the current position
    plot([q[1]], [q[2]], marker=(:hex, 10), xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
    
    # Run the MPC control optimization
    q_plan, v_plan, u_plan = run_mpc(q, v)
    
    # Draw the planned future states from the MPC optimization
    plot!(q_plan[1, :], q_plan[2, :], linewidth=5)
    
    # Apply the planned acceleration and simulate one step in time
    u = u_plan[:, 1]
    v += u * Δt
    q += v * Δt
end

INFO: Saved animation to /Users/rdeits/locomotion/explorations/Robocon2017Tutorial.jl/notebooks/tmp.gif


Plots.AnimatedGif("/Users/rdeits/locomotion/explorations/Robocon2017Tutorial.jl/notebooks/tmp.gif")